In [50]:
import pandas as pd
# from dataprep.datasets import load_dataset
# from dataprep.eda import create_report

df=pd.read_csv("train_titanic.csv")
# create_report(df).show()

# Ensambling
Agregare resultados de diferentes algoritmos o distintas instacias del mismo algoritmo, para obtener una mejor prediccion

# 1) Voting
 
# 2) Stacking
# 3) Bagging
# 4) Boosting

# Voting

In [51]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
X, y = load_iris(return_X_y=True)

In [52]:
y=np.where(y==2,1,0)
#transformamos el problema en binario

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y
)

In [53]:
from sklearn.ensemble import VotingClassifier

model1 = LogisticRegression()
model2 = SVC(probability=True)
model3=RandomForestClassifier()

In [54]:
model3

RandomForestClassifier()

In [55]:
voting_model = VotingClassifier(
    estimators=[("Logística", model1),
                ("SV Machine", model2),
                ("RandomForest", model3)],
    voting="soft")

voting_model = voting_model.fit(X_train,y_train)

In [56]:
voting_model.predict(X_test)

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0])

In [57]:
voting_model.get_params()

{'estimators': [('Logística', LogisticRegression()),
  ('SV Machine', SVC(probability=True)),
  ('RandomForest', RandomForestClassifier())],
 'flatten_transform': True,
 'n_jobs': None,
 'verbose': False,
 'voting': 'soft',
 'weights': None,
 'Logística': LogisticRegression(),
 'SV Machine': SVC(probability=True),
 'RandomForest': RandomForestClassifier(),
 'Logística__C': 1.0,
 'Logística__class_weight': None,
 'Logística__dual': False,
 'Logística__fit_intercept': True,
 'Logística__intercept_scaling': 1,
 'Logística__l1_ratio': None,
 'Logística__max_iter': 100,
 'Logística__multi_class': 'auto',
 'Logística__n_jobs': None,
 'Logística__penalty': 'l2',
 'Logística__random_state': None,
 'Logística__solver': 'lbfgs',
 'Logística__tol': 0.0001,
 'Logística__verbose': 0,
 'Logística__warm_start': False,
 'SV Machine__C': 1.0,
 'SV Machine__break_ties': False,
 'SV Machine__cache_size': 200,
 'SV Machine__class_weight': None,
 'SV Machine__coef0': 0.0,
 'SV Machine__decision_function_sh

In [58]:
yhat_test = voting_model.predict(X_test)
print("Acuracy:", accuracy_score(y_test,yhat_test))

Acuracy: 1.0


In [59]:
voting_model.predict_proba(X_test)
# probbilidad entre la mas alta de las dos colums, de ahi saca los 1 y 0

array([[9.99599871e-01, 4.00129361e-04],
       [9.75010151e-01, 2.49898488e-02],
       [9.99639445e-01, 3.60555133e-04],
       [9.99554270e-01, 4.45730278e-04],
       [9.96777925e-01, 3.22207469e-03],
       [7.78008111e-01, 2.21991889e-01],
       [1.43188898e-01, 8.56811102e-01],
       [9.99874568e-01, 1.25431865e-04],
       [9.92914357e-01, 7.08564285e-03],
       [1.01730234e-02, 9.89826977e-01],
       [2.37787023e-01, 7.62212977e-01],
       [9.99644313e-01, 3.55686770e-04],
       [9.99704767e-01, 2.95232824e-04],
       [9.99612382e-01, 3.87618253e-04],
       [9.99542352e-01, 4.57647513e-04],
       [8.12391982e-01, 1.87608018e-01],
       [2.46885306e-03, 9.97531147e-01],
       [1.15205499e-01, 8.84794501e-01],
       [9.99491040e-01, 5.08960365e-04],
       [9.80048476e-01, 1.99515239e-02],
       [9.99726336e-01, 2.73663865e-04],
       [9.99784610e-01, 2.15389852e-04],
       [9.09617238e-01, 9.03827621e-02],
       [1.45232063e-02, 9.85476794e-01],
       [4.022918

# Stacking

In [60]:
from sklearn.datasets import load_iris
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [61]:
model1=RandomForestClassifier(n_estimators=10, random_state=42)
model2=make_pipeline(StandardScaler(), SVC(random_state=42, probability=True))

In [62]:
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True, random_state=42))])

In [63]:
yhat1=model1.predict_proba(X_train)[:,1]
yhat2=model2.predict_proba(X_train)[:,1]

In [64]:
Y_predichas=pd.DataFrame({'yhat1':yhat1,'yhat2':yhat2})

In [65]:
final_estimator=LogisticRegression()
final_estimator.fit(Y_predichas, y_train)

LogisticRegression()

In [66]:
estimators = [("rf", model1), ("svr", model2)]

In [67]:
clf = StackingClassifier(estimators=estimators,
                         final_estimator=LogisticRegression()
                         )

In [68]:
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9736842105263158

In [69]:
clf.final_estimator_.coef_

array([[2.03065785, 3.12106841]])

In [70]:
clf.final_estimator_.intercept_

array([-2.86817877])

# Bagging